In [7]:


import cv2
import numpy as np
import os
import random
from tqdm import tqdm
import shutil
import yaml

In [6]:
def get_corners_from_yolo(bboxes, img_w, img_h):
    """YOLO 포맷을 코너 좌표로 변환합니다."""
    corners = []
    for bbox in bboxes:
        _, x_center, y_center, width, height = bbox
        w = width * img_w
        h = height * img_h
        x1 = (x_center * img_w) - (w / 2)
        y1 = (y_center * img_h) - (h / 2)
        x2 = x1 + w
        y2 = y1 + h
        corners.append([x1, y1, x2, y2])
    return np.array(corners)

def get_yolo_from_corners(corners, class_ids, img_w, img_h):
    """코너 좌표를 YOLO 포맷으로 변환합니다."""
    yolo_boxes = []
    for i, corner in enumerate(corners):
        x1, y1, x2, y2 = corner
        x1, x2 = min(x1, x2), max(x1, x2)
        y1, y2 = min(y1, y2), max(y1, y2)
        
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_w, x2), min(img_h, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        width = x2 - x1
        height = y2 - y1
        x_center = (x1 + width / 2) / img_w
        y_center = (y1 + height / 2) / img_h
        
        yolo_boxes.append([
            class_ids[i], 
            np.clip(x_center, 0, 1), 
            np.clip(y_center, 0, 1), 
            np.clip(width / img_w, 0, 1), 
            np.clip(height / img_h, 0, 1)
        ])
    return np.array(yolo_boxes)

def rotate_boxes(corners_4p, angle, cx, cy):
    """바운딩 박스를 회전시킵니다."""
    all_corners = corners_4p.reshape(-1, 2)
    all_corners_hom = np.hstack((all_corners, np.ones((all_corners.shape[0], 1))))

    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)

    rotated_all_corners = np.dot(M, all_corners_hom.T).T
    rotated_corners_4p = rotated_all_corners.reshape(-1, 4, 2)

    x_min = np.min(rotated_corners_4p[:, :, 0], axis=1)
    y_min = np.min(rotated_corners_4p[:, :, 1], axis=1)
    x_max = np.max(rotated_corners_4p[:, :, 0], axis=1)
    y_max = np.max(rotated_corners_4p[:, :, 1], axis=1)

    return np.column_stack((x_min, y_min, x_max, y_max))

# --- Augmentation Logic ---
def augment_image(image, bboxes):
    h, w = image.shape[:2]
    class_ids = bboxes[:, 0].copy()
    augmentations = {}
    
    # 1. 상하 반전
    img_vf = cv2.flip(image, 0)
    bboxes_vf = bboxes.copy()
    bboxes_vf[:, 2] = 1 - bboxes_vf[:, 2]
    augmentations['vf'] = (img_vf, bboxes_vf)
    
    # 2. 좌우 반전
    img_hf = cv2.flip(image, 1)
    bboxes_hf = bboxes.copy()
    bboxes_hf[:, 1] = 1 - bboxes_hf[:, 1]
    augmentations['hf'] = (img_hf, bboxes_hf)
    
    # 3. 밝기 조절
    brightness = random.uniform(0.5, 1.5)
    img_bright = cv2.convertScaleAbs(image, alpha=brightness, beta=0)
    augmentations['bright'] = (img_bright, bboxes.copy())
    
    # 4. 회전
    angle = random.uniform(-15, 15)
    rot_mat = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), borderMode=cv2.BORDER_REPLICATE)
    
    pixel_corners_2p = get_corners_from_yolo(bboxes, w, h)
    if pixel_corners_2p.size > 0:
        x1 = pixel_corners_2p[:, 0:1]
        y1 = pixel_corners_2p[:, 1:2]
        x2 = pixel_corners_2p[:, 2:3]
        y2 = pixel_corners_2p[:, 3:4]
        
        corners_4p = np.hstack([x1, y1, x2, y1, x1, y2, x2, y2]).reshape(-1, 4, 2)
        rotated_pixel_corners = rotate_boxes(corners_4p, angle, w/2, h/2)
        
        bboxes_rot = get_yolo_from_corners(rotated_pixel_corners, class_ids, w, h)
        if bboxes_rot.size > 0:
            augmentations['rot'] = (img_rot, bboxes_rot)

    return augmentations

# --- Main Function ---
def main():
    base_dir = r"C:\Empty_Parking\data\train"
    image_dir = os.path.join(base_dir, 'images')
    label_dir = os.path.join(base_dir, 'labels')

    if not os.path.exists(image_dir) or not os.path.exists(label_dir):
        print(f"Error: 디렉토리를 찾을 수 없습니다. 경로: {base_dir}")
        return

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png')) and not f.startswith('aug_')]
    print(f"총 {len(image_files)}개의 원본 이미지를 증강합니다.")

    for image_name in tqdm(image_files, desc='이미지 증강 중'):
        base_name, ext = os.path.splitext(image_name)
        label_name = f'{base_name}.txt'
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, label_name)

        if not os.path.exists(label_path):
            continue
            
        image = cv2.imread(image_path)
        if image is None:
            continue
            
        with open(label_path, 'r') as f:
            try:
                bboxes = np.array([line.strip().split() for line in f.readlines()], dtype=np.float32)
            except (ValueError, IndexError):
                continue
        
        if bboxes.ndim == 1:
            bboxes = np.array([bboxes])
            
        if bboxes.size == 0 or bboxes.shape[1] != 5:
            continue
            
        augmented_data = augment_image(image, bboxes)
        
        for aug_type, (aug_img, aug_bboxes) in augmented_data.items():
            if aug_bboxes.size == 0:
                continue
            aug_img_name = f'aug_{base_name}_{aug_type}{ext}'
            aug_label_name = f'aug_{base_name}_{aug_type}.txt'
            cv2.imwrite(os.path.join(image_dir, aug_img_name), aug_img)
            np.savetxt(os.path.join(label_dir, aug_label_name), aug_bboxes, fmt='%.6f')

    print(f'\n데이터 증강 완료. 총 이미지 수: {len(os.listdir(image_dir))}')

if __name__ == '__main__':
    main()


총 2775개의 원본 이미지를 증강합니다.


이미지 증강 중: 100%|██████████████████████████████████████████████████████████████| 2775/2775 [00:54<00:00, 51.20it/s]


데이터 증강 완료. 총 이미지 수: 13884


In [8]:
base_dir = r'C:\Empty_Parking'
source_data_dir = os.path.join(base_dir, 'data', 'train')
new_dataset_dir = os.path.join(base_dir, 'dataset_v4')

# --- 1. 
def split_dataset():
    print(" ")
    source_images = os.path.join(source_data_dir, 'images')
    source_labels = os.path.join(source_data_dir, 'labels')

    # 
    train_img_path = os.path.join(new_dataset_dir, 'train', 'images')
    train_lbl_path = os.path.join(new_dataset_dir, 'train', 'labels')
    valid_img_path = os.path.join(new_dataset_dir, 'valid', 'images')
    valid_lbl_path = os.path.join(new_dataset_dir, 'valid', 'labels')

    for path in [train_img_path, train_lbl_path, valid_img_path, valid_lbl_path]:
        os.makedirs(path, exist_ok=True)

    # 
    image_files = [f for f in os.listdir(source_images) if f.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(image_files)

    # 80/20 
    split_index = int(len(image_files) * 0.8)
    train_files = image_files[:split_index]
    valid_files = image_files[split_index:]

    # 
    def copy_files(file_list, dest_img_path, dest_lbl_path):
        for img_file in file_list:
            base_name = os.path.splitext(img_file)[0]
            lbl_file = f'{base_name}.txt'

            # 
            src_img = os.path.join(source_images, img_file)
            src_lbl = os.path.join(source_labels, lbl_file)

            # 
            dst_img = os.path.join(dest_img_path, img_file)
            dst_lbl = os.path.join(dest_lbl_path, lbl_file)

            if os.path.exists(src_lbl):
                shutil.copy(src_img, dst_img)
                shutil.copy(src_lbl, dst_lbl)
            else:
                print(f": {img_file}({lbl_file}) . ")

    print(f" {len(train_files)}...")
    copy_files(train_files, train_img_path, train_lbl_path)
    print(f" {len(valid_files)}...")
    copy_files(valid_files, valid_img_path, valid_lbl_path)
    print(" ")

# --- 2. YAML 
def create_yaml_file():
    print("data_v4.yaml ...")
    yaml_content = {
        'train': os.path.join(new_dataset_dir, 'train', 'images').replace('\\', '/'),
        'val': os.path.join(new_dataset_dir, 'valid', 'images').replace('\\', '/'),
        'nc': 1,
        'names': ['Occupied']
    }

    yaml_path = os.path.join(new_dataset_dir, 'data_v4.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)
    print(f"YAML : {yaml_path}")

if __name__ == '__main__':
    split_dataset()
    create_yaml_file()
    print("\n. ")


 
 11107...
 2777...
 
data_v4.yaml ...
YAML : C:\Empty_Parking\dataset_v4\data_v4.yaml

. 
